In [1]:
# we will need to create a transaction pool of incoming transactions, validate those transactions, and make them into a block
import hashlib, json, sys

def hashMe(msg=""):
    # For convenience, this is a helper function that wraps our hashing algorithm
    if type(msg)!= str:
        msg = json.dumps(msg, sort_keys=True)
        # If we don't sort keys, we can't guarantee repeatability
    if sys.version_info.major == 2:
        return unicode(hashlib.sha256(msg).hexdigest(), 'utf-8')
    else:
        return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

Next, we want to create a function to generate exchanges between John and Mary. We’ll indicate withdrawals with negative numbers, and deposits with positive numbers. We’ll construct our transactions to always be between the two users of our system, and make sure that the deposit is the same magnitude as the withdrawal- i.e. that we’re neither creating nor destroying money.

In [15]:
import random
random.seed(0)

def makeTransaction(maxValue=3):
    # This will create valid transactions in the range of (1, maxValue)
    sign = int(random.getrandbits(1))*2 - 1
    # This will randomly choose -1 or 1
    amount = random.randint(1, maxValue)
    johnPays = sign * amount
    maryPays = -1 * johnPays
    # By construction, this will always return transactions that respect the conservation of tokens
    # However, note that we have not done anything to check whether these overdraft an account
    return {u'John':johnPays, u'Mary':maryPays}


Now let’s create a large set of transactions, then chunk them into blocks.

In [16]:
txnBuffer = [makeTransaction() for i in range(30)]

In [4]:
def updateState(txn, state):
    # Inputs: txn, state: dictionaries keyed with account names, holding numeric values for transfer amount (txn) or account balance (state)
    # Returns: Updated state, with additional users added to state if necessary
    # NOTE: This does not not validate the transaction- just updates the state!
    state = state.copy()
    # As dictionaries are mutable, let's avoid any confusion by creating a working copy of the data
    for key in txn:
        if key in state.keys():
            state[key] += txn[key]
        else:
            state[key] = txn[key]
    return state

In [7]:
def isValidTxn(txn, state):
    # Assume that the transaction is a dictionary keyed by account names
    # Check that the sum of the deposites and withdrawals is 0
    if sum(txn.values())!=0:
        return False
    
    # Check that the transaction does not cause an overdraft
    for key in txn.keys():
        if key in state.keys():
            acctBalance = state[key]
        else:
            acctBalance = 0
        if (acctBalance + txn[key]) < 0:
            return False
    return True

In [8]:
state = {u'John':5,u'Mary':5}

print(isValidTxn({u'John': -3, u'Mary': 3},state))  # Basic transaction- this works great!
print(isValidTxn({u'John': -4, u'Mary': 3},state))  # But we can't create or destroy tokens!
print(isValidTxn({u'John': -6, u'Mary': 6},state))  # We also can't overdraft our account.
print(isValidTxn({u'John': -4, u'Mary': 2,'Lisa':2},state)) # Creating new users is valid
print(isValidTxn({u'John': -4, u'Mary': 3,'Lisa':2},state)) # But the same rules still apply!

True
False
False
True
False


In [10]:
state = {u'John':50, u'Mary':50}
# Define the initial state
genesisBlockTxns = [state]
genesisBlockContents = {u'blockNumber':0, u'parentHash':None, u'txnCount':1, u'txns':genesisBlockTxns}
genesisHash = hashMe(genesisBlockContents)
genesisBlock = {u'hash':genesisHash,u'contents':genesisBlockContents}
genesisBlockStr = json.dumps(genesisBlock, sort_keys=True)

In [11]:
chain = [genesisBlock]

In [12]:
def makeBlock(txns, chain):
    parentBlock = chain[-1]
    parentHash = parentBlock[u'hash']
    blockNumber = parentBlock[u'contents'][u'blockNumber'] + 1
    txnCount = len(txns)
    blockContents = {u'blockNumber':blockNumber, u'parentHash':parentHash, u'txnCount':len(txns), 'txns':txns}
    blockHash = hashMe(blockContents)
    block = {u'hash':blockHash, u'contents':blockContents}

    return block

In [17]:
blockSizeLimit = 5
# Arbitrary number of transactions per block
# this is chosen by the block miner, and can vary between blocks
while len(txnBuffer) > 0:
    bufferStartSize = len(txnBuffer)

    # Gather a set of valid transactions for inclusion
    txnList = []
    while (len(txnBuffer) > 0) & (len(txnList) < blockSizeLimit):
        newTxn = txnBuffer.pop()
        validTxn = isValidTxn(newTxn, state)
        # This will return False if txn is invalid
        if validTxn:
            txnList.append(newTxn)
            state = updateState(newTxn, state)
        else:
            print("ignored transaction")
            sys.stdout.flush()
            continue # This was an invalid transaction; ignore it and move on
    myBlock = makeBlock(txnList, chain)
    chain.append(myBlock)

In [18]:
chain[0]

{'hash': 'c33c4f338e7d9af4e27f5d9097c17811e8592653b6231056fe7bf9145f9dbb39',
 'contents': {'blockNumber': 0,
  'parentHash': None,
  'txnCount': 1,
  'txns': [{'John': 50, 'Mary': 50}]}}

In [19]:
chain[1]

{'hash': '1ac36126ff44b4340409e3ea0d7c4c9f7a745ea3a13824ccaa12d429edc6e336',
 'contents': {'blockNumber': 1,
  'parentHash': 'c33c4f338e7d9af4e27f5d9097c17811e8592653b6231056fe7bf9145f9dbb39',
  'txnCount': 5,
  'txns': [{'John': 3, 'Mary': -3},
   {'John': -1, 'Mary': 1},
   {'John': 3, 'Mary': -3},
   {'John': -2, 'Mary': 2},
   {'John': 3, 'Mary': -3}]}}

In [20]:
state

{'John': 72, 'Mary': 28}

In [21]:
len(chain)

7

In [22]:
chain[6]

{'hash': 'fa22ab8c250747b0c1154d4ec30c8d3408a23e0967a061b90cf3bf49e4e3d897',
 'contents': {'blockNumber': 6,
  'parentHash': '4cd838598c2153ca21fe15b491cdae82810599b141795e5c01b2cf3dbce87ab9',
  'txnCount': 5,
  'txns': [{'John': -2, 'Mary': 2},
   {'John': 3, 'Mary': -3},
   {'John': -2, 'Mary': 2},
   {'John': 2, 'Mary': -2},
   {'John': 2, 'Mary': -2}]}}

In [23]:
chain[5]

{'hash': '4cd838598c2153ca21fe15b491cdae82810599b141795e5c01b2cf3dbce87ab9',
 'contents': {'blockNumber': 5,
  'parentHash': 'f3115265bd16db0df3e672baeee7037767bded897d45b1fce82e0db31ad988ab',
  'txnCount': 5,
  'txns': [{'John': 1, 'Mary': -1},
   {'John': 1, 'Mary': -1},
   {'John': -3, 'Mary': 3},
   {'John': 2, 'Mary': -2},
   {'John': 2, 'Mary': -2}]}}

In [24]:
def checkBlockHash(block):
    # Raise an exception if the hash does not match the block contents
    expectedHash = hashMe(block['contents'])
    if block['hash']!=expectedHash:
        raise Exception('Hash does not match contents of block %s'%block['contents']['blockNumber'])
    return

In [25]:
def checkBlockValidity(block, parent, state):
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block has is valid for the block contents
    # - Block number increments the parent block number by 1
    # - Accurately references the parent block's hash
    parentNumber = parent['contents']['blockNumber']
    parentHash = parent['hash']
    blockNumber = block['contents']['blockNumber']

    # Check transaction validity; throw an error if an invalid transaction was found
    for txn in block['contents']['txns']:
        if isValidTxn(txn, state):
            state = updateState(txn, state)
        else:
            raise Exception('Invalid transaction in block %s: %s'%(blockNumber, txn))
    checkBlockHash(block) # Check hash integrity; raises error if inaccurate

    if blockNumber!=(parentNumber+1):
        raise Exception('Hash does not match contents of block %s'%blockNumber)
    if block['contents']['parentHash']!=parentHash:
        raise Exception('Parent hash not accurate at block %s'%blockNumber)
    return state

In [26]:
def checkChain(chain):
    # Work through the chain from the genesis block (which gets special treatment),
    # checking that all transactions are internally valid,
    # that the transactions do not cause an overdraft,
    # and that the blocks are linked by their hashes.
    # This returns the state as a dictionary of accounts and balances,
    # or returns False if an error was detected

    ##Data input processing: Make sure that our chain is a list of dicts
    if type(chain) == str:
        try:
            chain = json.loads(chain)
            assert(type(chain)==list)
        except: # This is a catch-all, admittedly crude
            return False
    elif type(chain)!=list:
        return False

    state = {}
    ## Prime the pump by checking the genesis block
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents

    for txn in chain[0]['contents']['txns']:
        state = updateState(txn, state)
    checkBlockHash(chain[0])
    parent = chain[0]

    ## Checking subsequent blocks: These additionally need to check
    # - the reference to the parent block's hash
    # - the validity of the block number
    for block in chain[1:]:
        state = checkBlockValidity(block, parent, state)
        parent = block
    return state

In [27]:
checkChain(chain)

{'John': 72, 'Mary': 28}

In [28]:
chainAsText = json.dumps(chain,sort_keys=True)
checkChain(chainAsText)

{'John': 72, 'Mary': 28}

In [29]:
import copy
nodeBchain = copy.copy(chain)
nodeBtxns = [makeTransaction() for i in range(5)]
newBlock = makeBlock(nodeBtxns, nodeBchain)

In [30]:
print("Blockchain on Node A is currently %s blocks long"%len(chain))
try:
    print("New Block Received; checking validity...")
    state = checkBlockValidity(newBlock, chain[-1], state)
    # Update the state- this will throw an error if the block is invalid
    chain.append(newBlock)
except:
    print("Invalid block; ignoring and waiting for the next block...")

print("Blockchain on Node A is not %s blocks long"%len(chain))

Blockchain on Node A is currently 7 blocks long
New Block Received; checking validity...
Blockchain on Node A is not 8 blocks long
